<a href="https://colab.research.google.com/github/farzanehnia/PYOMO/blob/main/Power_Generation_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
pip install -q pyomo

In [19]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd

In [20]:
#inputs
!rm inputs.xlsx

!wget --show-progress --continue -O /content/inputs.xlsx https://raw.github.com/farzanehnia/PYOMO/main/inputs.xlsx

dataGen = pd.read_excel('inputs.xlsx', sheet_name='gen')
dataLoad = pd.read_excel('inputs.xlsx', sheet_name='load')

Ng = len(dataGen)

import re
dataLoad['Load Demand'] = dataLoad['Load Demand'].astype('str').str.replace('\D', "").astype('float')
dataGen['Power Generation'] = dataGen['Power Generation'].astype('str').str.replace('\D', "").astype('float')


--2022-07-03 17:53:48--  https://raw.github.com/farzanehnia/PYOMO/main/inputs.xlsx
Resolving raw.github.com (raw.github.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.github.com (raw.github.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/farzanehnia/PYOMO/main/inputs.xlsx [following]
--2022-07-03 17:53:48--  https://raw.githubusercontent.com/farzanehnia/PYOMO/main/inputs.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9455 (9.2K) [application/octet-stream]
Saving to: ‘/content/inputs.xlsx’

/content/inputs.xls 100%[===================>]   9.23K  --.-KB/s    in 0s      

2022-07-03 17:53:48 (74.0 MB/s) - ‘/content/inputs.xls

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]


In [21]:

dataGen.head()
dataLoad['Load Demand'].loc[0]

50.0

In [22]:
#model

model = pyo.ConcreteModel()

model.Pg = pyo.Var(range(Ng), bounds=(0, None))
Pg = model.Pg

model.pprint()

1 Set Declarations
    Pg_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    Pg : Size=5, Index=Pg_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals

2 Declarations: Pg_index Pg


In [23]:
#constraints
Pg_sum = sum([Pg[g] for g in dataGen.id])
demand_sum = dataLoad['Load Demand'].sum()


model.balance = pyo.Constraint(expr = Pg_sum == demand_sum)

model.cond = pyo.Constraint(expr = Pg[0]+Pg[3] >= dataLoad['Load Demand'].iloc[0])

model.limits = pyo.ConstraintList()
for g in dataGen.id:
  model.limits.add(expr = Pg[g] <= dataGen['Power Generation'].iloc[g])

In [24]:
#Obj func

model.obj = pyo.Objective(expr = sum([Pg[g]*dataGen.Cost[g] for g in dataGen.id]))


In [25]:
#Cost sumation practice
print(sum([Pg[g]*dataGen['Cost'].loc[g] for g in dataGen.id]))
print(sum([Pg[g]*dataGen.Cost[g] for g in dataGen.id]))

0.1*Pg[0] + 0.05*Pg[1] + 0.3*Pg[2] + 0.4*Pg[3] + 0.01*Pg[4]
0.1*Pg[0] + 0.05*Pg[1] + 0.3*Pg[2] + 0.4*Pg[3] + 0.01*Pg[4]


In [26]:
!apt install glpk-utils
!pip install glpk


Reading package lists... Done
Building dependency tree       
Reading state information... Done
glpk-utils is already the newest version (4.65-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached glpk-0.4.6.tar.gz (153 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  ERROR: Failed building wheel for glpk
Failed to build glpk
ERROR: Could not build wheels for glpk which use PEP 517 and cannot be installed directly


In [27]:
opt = SolverFactory('glpk') #GLPK,cbc
results = opt.solve(model)

In [28]:
#print(results)
dataGen['Pg'] = [pyo.value(Pg[g]) for g in dataGen.id]
dataGen.head()

,id,Cost,Power Generation,Pg
0,0,0.10,20.0,20.0
1,1,0.05,10.0,10.0
2,2,0.30,40.0,35.0
3,3,0.40,50.0,30.0
4,4,0.01,5.0,5.0


In [29]:
model.Pg.pprint()

Pg : Size=5, Index=Pg_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  20.0 :  None : False : False :  Reals
      1 :     0 :  10.0 :  None : False : False :  Reals
      2 :     0 :  35.0 :  None : False : False :  Reals
      3 :     0 :  30.0 :  None : False : False :  Reals
      4 :     0 :   5.0 :  None : False : False :  Reals


In [30]:
model.cond.pprint()

cond : Size=1, Index=None, Active=True
    Key  : Lower : Body          : Upper : Active
    None :  50.0 : Pg[0] + Pg[3] :  +Inf :   True
